In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_assets_info = pd.read_csv('/content/drive/MyDrive/RS Metrics - Datasets/nippon_steel_asset_info.csv')

In [ ]:
df_assets_info.drop(['Latitude' ,'Longitude'], axis = 1, inplace = True)

In [ ]:
df_assets_info.isna().sum()

Location ID            0
LocationName           0
LocationTypeGeneral    0
Sector                 0
Industry               0
SubIndustry            0
CountryName            0
dtype: int64

In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=34a432b4e66932a062d9b72a5ad1fe092a3f7f4b6b49493a8330da7f6f1920e6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('RSMetrics').getOrCreate()

In [ ]:
spark

In [ ]:
py_info = spark.createDataFrame(df_assets_info)

In [ ]:
py_info.show()

+-----------+--------------------+-------------------+---------+---------------+-----------+--------------------+
|Location ID|        LocationName|LocationTypeGeneral|   Sector|       Industry|SubIndustry|         CountryName|
+-----------+--------------------+-------------------+---------+---------------+-----------+--------------------+
|   ESG_7306|Sumitomo Osaka Of...|             Office|Materials|Metals & Mining|      Steel|               Japan|
|   ESG_7322|Hyuga Smelting Plant|              Plant|Materials|Metals & Mining|      Steel|               Japan|
|   ESG_8033|Tek and Kote Stee...|              Plant|Materials|Metals & Mining|      Steel|United States of ...|
|   ESG_8275|Usinas Steel Belo...|              Plant|Materials|Metals & Mining|      Steel|              Brazil|
|   ESG_8276|Usiminas Ipatinga...|              Plant|Materials|Metals & Mining|      Steel|              Brazil|
|   ESG_8277|Vallourec Solucoe...|              Plant|Materials|Metals & Mining|      St

In [ ]:
py_2018 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2018.csv', header = True, inferSchema=True)
py_2019 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2019.csv', header = True, inferSchema=True)
py_2020 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2020.csv', header = True, inferSchema=True)
py_2021 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2021.csv', header = True, inferSchema=True)
py_2022 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2022.csv', header = True, inferSchema=True)
py_2023 = spark.read.csv('/content/drive/MyDrive/RS Metrics - Datasets/Merged2023.csv', header = True, inferSchema=True)

In [ ]:
py_rainfall = py_2018.union(py_2019).union(py_2020).union(py_2021).union(py_2022).union(py_2023)

In [ ]:
py_rainfall.count()

69090

In [ ]:
py_rainfall.show()

+---------+-----------+-----------------------+-----------+-----------+
|Year_Week|Location ID|Precipitation (mm/week)|   Latitude|  Longitude|
+---------+-----------+-----------------------+-----------+-----------+
|  2018-01|   ESG_7306|     162.92968683866604|34.69200035|135.4991346|
|  2018-02|   ESG_7306|      97.95245834716468|34.69200035|135.4991346|
|  2018-03|   ESG_7306|     12.509563760371773|34.69200035|135.4991346|
|  2018-04|   ESG_7306|     24.143830504881983|34.69200035|135.4991346|
|  2018-05|   ESG_7306|     114.55654900699268|34.69200035|135.4991346|
|  2018-06|   ESG_7306|      42.37310381346912|34.69200035|135.4991346|
|  2018-07|   ESG_7306|      72.23472046044962|34.69200035|135.4991346|
|  2018-08|   ESG_7306|     134.87847853397915|34.69200035|135.4991346|
|  2018-09|   ESG_7306|      208.2857495759924|34.69200035|135.4991346|
|  2018-10|   ESG_7306|     168.49420713072317|34.69200035|135.4991346|
|  2018-11|   ESG_7306|      386.1015191670762|34.69200035|135.4

In [ ]:
#df_rainfall = py_rainfall.toPandas()

In [ ]:
#df_rainfall.to_csv("/content/drive/MyDrive/RS Metrics - Datasets/All_Merged.csv", index = False)

In [ ]:
#columns_to_drop = ['Latitude' ,'Longitude']
#py_rainfall = py_rainfall.drop(*columns_to_drop)

In [ ]:
py_rainfall = py_rainfall.withColumnRenamed("Location ID", "Rainfall Location ID")

In [ ]:
py_rainfall = py_rainfall.join(py_info, py_rainfall['Rainfall Location ID']==py_info['Location ID'])

In [ ]:
#py_rainfall = py_rainfall.drop('Location')

In [ ]:
df_rainfall = py_rainfall.toPandas()

In [ ]:
df_rainfall.to_csv("/content/drive/MyDrive/RS Metrics - Datasets/Rainfall Data All Merged.csv", index = False)

In [ ]:
py_rainfall = py_rainfall.drop('Location ID')

In [ ]:
py_rainfall.printSchema()

root
 |-- Year_Week: string (nullable = true)
 |-- Rainfall Location ID: string (nullable = true)
 |-- Precipitation (mm/week): double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- LocationName: string (nullable = true)
 |-- LocationTypeGeneral: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- SubIndustry: string (nullable = true)
 |-- CountryName: string (nullable = true)



In [ ]:
distinct_country_names = py_rainfall.select("CountryName").distinct()
distinct_country_names.show(21)

+--------------------+
|         CountryName|
+--------------------+
|              Sweden|
|            Malaysia|
|             Germany|
|   Brunei Darussalam|
|               India|
|               China|
|            Thailand|
|United States of ...|
|              Mexico|
|           Indonesia|
|        Saudi Arabia|
|United Arab Emirates|
|United Kingdom of...|
|              Brazil|
|            Viet Nam|
|               Japan|
|           Australia|
|           Singapore|
|             Nigeria|
|              Canada|
|        South Africa|
+--------------------+



In [ ]:
py_rainfall.select('Industry').distinct().show()

+--------------------+
|            Industry|
+--------------------+
|     Metals & Mining|
|Professional Serv...|
|           Chemicals|
|Air Freight & Log...|
|Oil, Gas & Consum...|
+--------------------+



In [ ]:
df_rainfall = py_rainfall.toPandas()

In [ ]:
# Split 'Year_Week' into 'Year' and 'Week' columns
df_rainfall[['Year', 'Week']] = df_rainfall['Year_Week'].str.split('-', expand=True)
df_rainfall.head()

,Year_Week,Rainfall Location ID,Precipitation (mm/week),Latitude,Longitude,LocationName,LocationTypeGeneral,Sector,Industry,SubIndustry,CountryName,Year,Week
0,2023-36,ESG_7306,16.611677,34.692,135.499135,Sumitomo Osaka Office,Office,Materials,Metals & Mining,Steel,Japan,2023,36
1,2023-35,ESG_7306,10.962413,34.692,135.499135,Sumitomo Osaka Office,Office,Materials,Metals & Mining,Steel,Japan,2023,35
2,2023-34,ESG_7306,118.319349,34.692,135.499135,Sumitomo Osaka Office,Office,Materials,Metals & Mining,Steel,Japan,2023,34
3,2023-33,ESG_7306,243.764635,34.692,135.499135,Sumitomo Osaka Office,Office,Materials,Metals & Mining,Steel,Japan,2023,33
4,2023-32,ESG_7306,18.451549,34.692,135.499135,Sumitomo Osaka Office,Office,Materials,Metals & Mining,Steel,Japan,2023,32


In [ ]:
# Show the resulting DataFrame
df_rainfall.to_csv('/content/drive/MyDrive/RS Metrics - Datasets/RainfallwithMonth.csv', index = False)

In [ ]:
df_rainfall.columns

Index(['Year_Week', 'Rainfall Location ID', 'Precipitation (mm/week)',
       'Latitude', 'Longitude', 'LocationName', 'LocationTypeGeneral',
       'Sector', 'Industry', 'SubIndustry', 'CountryName', 'Year', 'Week'],
      dtype='object')